<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/inria-25-alpha.png" style="display:inline"></span><br/>

# La librairie `asyncio`

* boucle d'événements

* synchronisation: `Queue`, `Lock` et `Semaphore`

* interaction avec les processus:
  * package `asyncio.subprocess`

* `Transport` et `Protocol` / callback
* `Streams` / API orientées coroutine

# Utilitaire

In [ ]:
import asyncio

def reset_loop():
    asyncio.set_event_loop(asyncio.new_event_loop())

In [ ]:
from asynchelpers import start_time, sequence

# synchronisation avec une queue

In [ ]:
# pour être robuste si on re-exécute ce fragment
reset_loop()

In [ ]:
queue = asyncio.Queue(maxsize=1)

In [ ]:
async def producer(queue):
    count = 1
    while True:
        await queue.put(f'tick{count}')
        count += 1
        await asyncio.sleep(1)

In [ ]:
async def consumer(queue):
    while True:
        received = await queue.get()
        print(f"got {received}")
        

In [ ]:
# on ajoute les coroutines dans la boucle
asyncio.ensure_future(producer(queue))
asyncio.ensure_future(consumer(queue))

In [ ]:
try:
    asyncio.get_event_loop().run_forever()
except:
    print("bye")

# Limiter le parallèlisme avec `Queue`

In [ ]:
reset_loop()

In [ ]:
window = asyncio.Queue(maxsize = 4)

In [ ]:
async def job(i):
    # prendre un jeton dans la queue
    await window.put(None)
    # ps tout le monde la même durée
    duration = (i % 3) + 1
    await sequence(f"job{i} - duration {duration}", delay=duration)
    # libérer le jeton
    await window.get()

In [ ]:
for i in range(8):
    asyncio.ensure_future(job(i))

start_time()
try:
    asyncio.get_event_loop().run_forever()
except:
    print('bye')

Déroulement des jobs en fonction du temps

|     | j0 (1) | j1(2) | j2(3) | j3(1) | j4(2) | j5(3) | j6(1) | j7(2) |
|-----|--------|-------|-------|-------|-------|-------|-------|-------|
| 0-1 | `*`    | `*`   | `*`   | `*`   |       |       |       |       |
| 1-2 |        | `*`   | `*`   |       | `*`   | `*`   |       |       |
| 2-3 |        |       | `*`   |       | `*`   | `*`   | `*`   |       |
| 3-4 |        |       |       |       |       | `*`   |       | `*`   |
| 4-5 |        |       |       |       |       |       |       | `*`   |